# Chess Board Detection and Analysis
Computer vision for chess.com with Raspberry Pi Pico Control

## Import Libraries

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import copy
import os
import time
import keyboard
import serial

# Pico configuration
PICO_PORT = 'COM15'
PICO_BAUD_RATE = 115200

## Chess Piece Color Detection Functions

In [ ]:
def detect_chess_pieces(lower_h, lower_s, lower_v, upper_h, upper_s, upper_v, image_path):
    """Detect chess pieces of a specific color in an image"""
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not read image {image_path}")
        return []
    
    image_copy = image.copy()
    
    # Reduce image size for faster processing
    reduction_ratio = 0.4
    resized_image = cv2.resize(image_copy, (0, 0), fx=reduction_ratio, fy=reduction_ratio)
    
    # Convert to HSV color space
    hsv_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2HSV)
    
    # Define color range
    lower_color = np.array([lower_h, lower_s, lower_v])
    upper_color = np.array([upper_h, upper_s, upper_v])
    
    # Create mask
    mask = cv2.inRange(hsv_image, lower_color, upper_color)
    
    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Extract piece centers
    piece_coordinates = []
    for contour in contours:
        moments = cv2.moments(contour)
        if moments["m00"] > 250:  # Filter by area
            cx = int(moments["m10"] / moments["m00"])
            cy = int(moments["m01"] / moments["m00"])
            piece_coordinates.append((cx, cy))
            
            # Draw circle on image
            cv2.circle(resized_image, (cx, cy), 5, (0, 0, 255), -1)
    
    print(f"Detected {len(piece_coordinates)} pieces")
    return piece_coordinates

def calibrate_color():
    """Interactive color calibration tool"""
    def mouse_callback(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            hsv = cv2.cvtColor(param['image'], cv2.COLOR_BGR2HSV)
            h, s, v = hsv[y, x]
            print(f"Selected color - H: {h}, S: {s}, V: {v}")
    
    image_path = input("Enter image path: ")
    image = cv2.imread(image_path)
    
    cv2.namedWindow('Color Calibration')
    cv2.setMouseCallback('Color Calibration', mouse_callback, {'image': image})
    cv2.imshow('Color Calibration', image)
    
    print("Click on colors to get H,S,V values. Press ESC to exit.")
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## Chess Board Grid Detection

In [ ]:
def detect_board_corners(image_path):
    """Detect the four corners of the chess board"""
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Find edges
    edges = cv2.Canny(gray, 50, 150)
    
    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Find largest contour (should be the board)
    largest_contour = max(contours, key=cv2.contourArea)
    
    # Approximate contour to get corners
    epsilon = 0.02 * cv2.arcLength(largest_contour, True)
    corners = cv2.approxPolyDP(largest_contour, epsilon, True)
    
    return corners.reshape(-1, 2)

def create_board_grid(corners, image_path):
    """Create a grid representation of the board"""
    image = cv2.imread(image_path)
    
    # Sort corners
    corners = corners[np.argsort(corners[:, 1])]
    
    # Calculate cell centers for each square
    cell_coords = []
    rows = 8
    cols = 8
    
    for row in range(rows):
        row_coords = []
        for col in range(cols):
            # Interpolate position
            x = corners[0, 0] + (corners[1, 0] - corners[0, 0]) * col / cols
            y = corners[0, 1] + (corners[2, 1] - corners[0, 1]) * row / rows
            row_coords.append((x, y))
        cell_coords.append(row_coords)
    
    return np.array(cell_coords)

def map_piece_to_square(piece_coords, grid_coords):
    """Map detected piece coordinates to chess squares"""
    squares = {}
    
    for piece in piece_coords:
        # Find nearest grid point
        distances = np.sqrt((grid_coords[:, :, 0] - piece[0])**2 + (grid_coords[:, :, 1] - piece[1])**2)
        row, col = np.unravel_index(distances.argmin(), distances.shape)
        
        # Convert to chess notation
        file_letter = chr(ord('a') + col)
        rank_number = 8 - row
        
        squares[f"{file_letter}{rank_number}"] = piece
    
    return squares

In [ ]:
def send_piece_info_to_pico(piece_start, piece_end, is_capture=False):
    """Send piece movement information to Pico"""
    try:
        pico = serial.Serial(PICO_PORT, PICO_BAUD_RATE, timeout=1)
        time.sleep(2)
        
        move_type = "1" if is_capture else "0"
        message = f"{piece_start} {piece_end} {move_type}\n"
        
        pico.write(message.encode())
        print(f"Sent to Pico: {message.strip()}")
        
        response = pico.readline().decode().strip()
        if response:
            print(f"Pico response: {response}")
        
        pico.close()
        return True
    except Exception as e:
        print(f"Error sending to Pico: {e}")
        return False

def main_chess_detection():
    """Main chess detection workflow"""
    print("="*50)
    print("Chess Board Detection System (Pico Control)")
    print("="*50)
    
    image_path = input("Enter path to board image: ")
    
    # Detect board corners
    corners = detect_board_corners(image_path)
    grid = create_board_grid(corners, image_path)
    
    # Detect pieces
    pieces = detect_chess_pieces(0, 100, 100, 10, 255, 255, image_path)
    
    # Map to squares
    piece_squares = map_piece_to_square(pieces, grid)
    print(f"\nDetected pieces on squares:")
    for square, coord in piece_squares.items():
        print(f"  {square}: {coord}")
    
    # Get move from user
    print("\nEnter move (e.g., e2 e4):")
    start = input("Start square: ").lower()
    end = input("End square: ").lower()
    
    # Send to Pico
    send_piece_info_to_pico(start, end)

# Uncomment to run
# main_chess_detection()